Link to my repository:

https://github.com/ryangreen6/Time-Series-Plots-RG



In [1]:
# Import necessary packages

import xarray as xr
import matplotlib.pyplot as plt
import intake
import s3fs
import pandas as pd
import numpy as np

In [2]:
# Open the CMIP6 data catalog, store as a variable
catalog = intake.open_esm_datastore('https://cmip6-pds.s3.amazonaws.com/pangeo-cmip6.json')

In [3]:
# Specify search terms to query catalog for CanESM5 data
# activity_id: which project do you want? CMIP = historical data, ScenarioMIP = future projections
activity_ids = ['ScenarioMIP', 'CMIP'] 

# source_id: which model do you want? 
source_id = ['CanESM5']

# experiment_id: what experimental configuration do you want? Here we want historical and the four main SSPs
experiment_ids = ['historical', 'ssp245', 'ssp585']

# member_id: which ensemble member do you want? Here we want r10i1p1f1
member_id = 'r10i1p1f1'

# table_id: which part of the Earth system and time resolution do you want? Here we want monthly atmosphere data
table_id = 'Amon' 

# variable_id: which climate variable do you want? Here we want surface air temperature
variable_id = 'tas' 

In [4]:
# source_id: Second model
source_id_2 = ['MIROC6']

# member_id: 
member_id_2 = 'r6i1p1f1'

# table_id: 
table_id_2 = 'Omon' 

# variable_id: sea surface temperature
variable_id_2 = 'tos' 

In [5]:
# Search through catalog, store results in "res" variable
res = catalog.search(activity_id = activity_ids, 
                     source_id = source_id, 
                     experiment_id = experiment_ids, 
                     member_id = member_id, 
                     table_id = table_id, 
                     variable_id = variable_id)

res_2 = catalog.search(activity_id = activity_ids, 
                       source_id = source_id_2, 
                       experiment_id = experiment_ids,
                       member_id = member_id_2, 
                       table_id = table_id_2, 
                       variable_id = variable_id_2)

# Display data frame associated with results
display(res.df)

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,CCCma,CanESM5,historical,r10i1p1f1,Amon,tas,gn,s3://cmip6-pds/CMIP6/CMIP/CCCma/CanESM5/histor...,NaN,20190429
1,ScenarioMIP,CCCma,CanESM5,ssp585,r10i1p1f1,Amon,tas,gn,s3://cmip6-pds/CMIP6/ScenarioMIP/CCCma/CanESM5...,NaN,20190429
2,ScenarioMIP,CCCma,CanESM5,ssp245,r10i1p1f1,Amon,tas,gn,s3://cmip6-pds/CMIP6/ScenarioMIP/CCCma/CanESM5...,NaN,20190429


In [6]:
# Read in the historical data file
hist_data = xr.open_zarr(res.df['zstore'][0], storage_options={'anon': True})
hist_data_2 = xr.open_zarr(res_2.df['zstore'][0], storage_options={'anon': True})

# Read in the SSP245 data file
ssp245_data = xr.open_zarr(res.df['zstore'][1], storage_options={'anon': True})
ssp245_data_2 = xr.open_zarr(res_2.df['zstore'][1], storage_options={'anon': True})
# Read in SSP585 file
ssp585_data = xr.open_zarr(res.df['zstore'][2], storage_options={'anon': True})
ssp585_data_2 = xr.open_zarr(res_2.df['zstore'][2], storage_options={'anon': True})

In [7]:
# Concatenate historical and future projection data
canesm5_245 = xr.concat([hist_data, ssp245_data], dim="time")
miroc_245 = xr.concat([hist_data_2, ssp245_data_2], dim="time")

canesm5_585 = xr.concat([hist_data, ssp585_data], dim="time")
miroc_585 = xr.concat([hist_data_2, ssp585_data_2], dim="time")

In [8]:
# Convert time to datetime64 format
time_245 = canesm5_245.time.astype('datetime64[ns]')
time_245_2 = miroc_245.time.astype('datetime64[ns]')

time_585 = canesm5_585.time.astype('datetime64[ns]')
time_585_2 = miroc_585.time.astype('datetime64[ns]')

In [9]:
# Define min/max bounds for region of interest (Aegean Sea)
lat_min, lat_max = 34, 42
lon_min, lon_max = 19, 30

# Define logical mask: True when lat/lon inside the valid ranges, False elsewhere
tas_aeg_lat_245 = (canesm5_245.lat >= lat_min) & (canesm5_245.lat <= lat_max)
tas_aeg_lon_245 = (canesm5_245.lon >= lon_min) & (canesm5_245.lon <= lon_max)

# Find points where the mask value is True, drop all other points
tas_aeg_245 = canesm5_245.where(tas_aeg_lat_245 & tas_aeg_lon_245, drop=True)

# Average over lat, lon dimensions to get a time series
tas_aeg_245 = tas_aeg_245.mean(dim=["lat", "lon"])

In [10]:
tas_aeg_lat_245_m = (miroc_245.latitude >= lat_min) & (miroc_245.latitude <= lat_max)
tas_aeg_lon_245_m = (miroc_245.longitude >= lon_min) & (miroc_245.longitude <= lon_max)

# Find points where the mask value is True, drop all other points
tas_aeg_245_m = miroc_245.where(tas_aeg_lat_245_m & tas_aeg_lon_245_m, drop=True)

# Average over lat, lon dimensions to get a time series
tas_aeg_245 = tas_aeg_245_m.mean(dim=["lat", "lon"])

ValueError: Dataset does not contain the dimensions: ['lon', 'lat']

In [ ]:
tas_aeg_lat_245_2 = (canesm5_245_2.latitude >= lat_min) & (canesm5_245_2.latitude <= lat_max)
tas_aeg_lon_245_2 = (canesm5_245_2.longitude >= lon_min) & (canesm5_245_2.longitude <= lon_max)

# Find points where the mask value is True, drop all other points
tas_aeg_245_2 = canesm5_245_2.where(tas_aeg_lat_245_2 & tas_aeg_lon_245_2, drop=True)

# Average over lat, lon dimensions to get a time series
tas_aeg_245_2 = tas_aeg_245_2.mean(dim=["latitude", "longitude"])

In [ ]:
tas_aeg_lat_585_2 = (canesm5_585_2.latitude >= lat_min) & (canesm5_585_2.latitude <= lat_max)
tas_aeg_lon_585_2 = (canesm5_585_2.longitude >= lon_min) & (canesm5_585_2.longitude <= lon_max)

# Find points where the mask value is True, drop all other points
tas_aeg_585_2 = canesm5_585_2.where(tas_aeg_lat_585_2 & tas_aeg_lon_585_2, drop=True)

# Average over lat, lon dimensions to get a time series
tas_aeg_585_2 = tas_aeg_585_2.mean(dim=["latitude", "longitude"])

For this assignment I chose the Aegean Sea. This is a Medterranean climate characterized by hot summers and mild, wet winters. Vegetation is mainly medterranean forest, sparse woodlands, and scrubland.

Climate change impacts that could effect this region are drought due to changed precipitation patterns, marine heat waves, and of course sea level rise. 

In [ ]:
# Checking contents of the xarray
tas_aeg

In [ ]:
# Plotting temperatures
fig, ax = plt.subplots(figsize=(20, 8))
ax.plot(time, tas_aeg.tas, label='Near-Surface Air Temperature', color='b')
ax.set_title("Time Series of Aegean Sea Near-Surface Air Temperature (1850 to 2100) ", fontsize=20)
ax.set_xlabel("Time", fontsize=20)
ax.set_ylabel("Temperature (K)", fontsize=20)
ax.legend(fontsize=20)
ax.grid()
plt.show()

In [ ]:
# Select a time period of interest -- two time periods for this assignment
tas_aeg_2100 = tas_aeg.sel(time=slice('1850-01-16','2100-12-31'))
tas_aeg_2015 = tas_aeg.sel(time=slice('1850-01-16','2015-12-31'))

In [ ]:
# Dropping the time_bnds variable 
tas_aeg_2100 = tas_aeg_2100.drop_vars("time_bnds")
tas_aeg_2015 = tas_aeg_2015.drop_vars("time_bnds")

In [ ]:
# confirming dropped variable
tas_aeg_2015
tas_aeg_2100

In [ ]:
# calculating annual mean 
annual_mean_2100 = tas_aeg_2100.groupby('time.year').mean()
annual_mean_2015 = tas_aeg_2015.groupby('time.year').mean()

In [ ]:
# converting to celsius from Kelvin
annual_mean_2100 = annual_mean_2100 - 273.15
annual_mean_2015 = annual_mean_2015 - 273.15

In [ ]:
# Adding this cell to note that I wasn't able to get this chunk from the tutorial working:

# # Calculate best-fit parameters for the linear polynomial fit of temperature to year
# x = np.polyfit(annual_mean.year, annual_mean, 1)

# # Generate a polynomial object using those best-fit parameters
# trend_line = np.poly1d(x)  


# The cell below is my workaround

In [ ]:
# Pull out the DataArrays (variable name “tas”)
da_2100 = annual_mean_2100["tas"]
da_2015 = annual_mean_2015["tas"]

# Grab arrays for 2100 and 2015
years_2100 = da_2100["year"].values
temps_2100 = da_2100.values
years_2015 = da_2015["year"].values
temps_2015 = da_2015.values

# Fitting separate trend lines
coeffs_2100     = np.polyfit(years_2100, temps_2100, 1)
trend_line_2100 = np.poly1d(coeffs_2100)

coeffs_2015     = np.polyfit(years_2015, temps_2015, 1)
trend_line_2015 = np.poly1d(coeffs_2015)

# evaluate the fits
fit_2100 = trend_line_2100(years_2100)
fit_2015 = trend_line_2015(years_2015)

In [ ]:
# plotting the two time series and their trend lines

fig, ax = plt.subplots(figsize=(16, 3.5))
ax.plot(years_2100, temps_2100, label="Annual Average Temperature")
ax.plot(years_2100, fit_2100, linestyle="--", label=f"Trend Line 1850-2100 (Slope = {coeffs_2100[0]:.4f} °C/year)")
ax.plot(years_2015, fit_2015, linestyle="-", label=f"Trend Line 1850-2015 (Slope = {coeffs_2015[0]:.4f} °C/year)")
ax.set_title("Regional Average Surface Air Temp of the Aegean Sea (1850 to 2100)")
ax.set_xlabel("Year")
ax.set_ylabel("Temperature (°C)")
ax.legend()
plt.show()

Over these two time periods, the temperature is certainly changing. From 1850 to 2015, the change is relatively insignificant. However from 1850 to the projected temperatures in 2100, the change is extremely significant. 

The results of this plot are alarming, to say the least. This model indicates a change of nearly 2 degrees celsius by 2050, which is significant especially in a region with mild Mediterranean weather. Other regions could experience much higher increases in temperature I would imagine. 

This increase in temperature would effect ocean life like kelp would be effected, and probably trigger mass die-offs. Desertification may also be a concern; 50% of the land area of Crete is already at risk of desertification. 

As for human systems, this temperature increase would effect crop yields for crops like olives, grapes, and citrus, and would likely shift their ideal growing regions further north up the Adriatic. Drought would also effect these yields, and would directly effect potable water availability for populated areas. Heat waves could also effect tourism; just last summer several tourists died due to the heat on the islands (although I believe these guys were hiking without water, always bring water). Islands would be effected by sea level rise, and coastal populations may be displaced. Low elevation archaeological/heritage sites could also be effected by sea level rise.